In [5]:
import time
import random
import math

people = [('Seymour','BOS'),
          ('Franny','DAL'),
          ('Zooey','CAK'),
          ('Walt','MIA'),
          ('Buddy','ORD'),
          ('Les','OMA')]
# Laguardia
destination='LGA'


flights={}
# 
for line in open('schedule.txt'):
  origin,dest,depart,arrive,price=line.strip().split(',')
  flights.setdefault((origin,dest),[])

  # Add details to the list of possible flights
  flights[(origin,dest)].append((depart,arrive,int(price)))

def getminutes(t):
  x=time.strptime(t,'%H:%M')
  return x[3]*60+x[4]

def printschedule(r):
  for d in range(int(len(r)/2)):
    name=people[d][0]
    origin=people[d][1]
    out=flights[(origin,destination)][int(r[d*2])]
    ret=flights[(destination,origin)][int(r[d*2+1])]
    print('%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (name,origin,out[0],out[1],out[2],ret[0],ret[1],ret[2]))
    
def schedulecost(sol):
  totalprice=0
  latestarrival=0
  earliestdep=24*60

  for d in range(int(len(sol)/2)):
    # Get the inbound and outbound flights
    origin=people[d][1]
    outbound=flights[(origin,destination)][int(sol[d*2])]
    returnf=flights[(destination,origin)][int(sol[d*2+1])]
    
    # Total price is the price of all outbound and return flights
    totalprice+=outbound[2]
    totalprice+=returnf[2]
    
    # Track the latest arrival and earliest departure
    if latestarrival<getminutes(outbound[1]): latestarrival=getminutes(outbound[1])
    if earliestdep>getminutes(returnf[0]): earliestdep=getminutes(returnf[0])
    #print('last %10s' % latestarrival)    
  
  # Every person must wait at the airport until the latest person arrives.
  # They also must arrive at the same time and wait for their flights.
  totalwait=0  
  for d in range(int(len(sol)/2)):
    origin=people[d][1]
    outbound=flights[(origin,destination)][int(sol[d*2])]
    returnf=flights[(destination,origin)][int(sol[d*2+1])]
    totalwait+=latestarrival-getminutes(outbound[1])
    totalwait+=getminutes(returnf[0])-earliestdep  

  # Does this solution require an extra day of car rental? That'll be $50!
  if latestarrival>earliestdep: totalprice+=50
  #print(totalprice)
  return totalprice+totalwait
    
def randomoptimize(domain,costf):
  best=999999999
  bestr=None
  for i in range(0,4000):
    # Create a random solution
    r=[float(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
    
    # Get the cost
    cost=costf(r)
    
    # Compare it to the best one so far
    if cost<best:
      best=cost
      bestr=r 
  return r

def hillclimb(domain,costf):
  # Create a random solution
  sol=[random.randint(domain[i][0],domain[i][1])
      for i in range(len(domain))]
  # Main loop
  while 1:
    # Create list of neighboring solutions
    neighbors=[]
    
    for j in range(len(domain)):
      # One away in each direction
      if sol[j]>domain[j][0]:
        neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])
      if sol[j]<domain[j][1]:
        neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:])

    # See what the best solution amongst the neighbors is
    current=costf(sol)
    best=current
    for j in range(len(neighbors)):
      cost=costf(neighbors[j])
      if cost<best:
        best=cost
        sol=neighbors[j]

    # If there's no improvement, then we've reached the top
    if best==current:
      break
  return sol

def annealingoptimize(domain,costf,T=10000.0,cool=0.95,step=1):
  # Initialize the values randomly
  vec=[float(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
  
  while T>0.1:
    # Choose one of the indices
    i=random.randint(0,len(domain)-1)

    # Choose a direction to change it
    dir=random.randint(-step,step)

    # Create a new list with one of the values changed
    vecb=vec[:]
    vecb[i]+=dir
    if vecb[i]<domain[i][0]: vecb[i]=domain[i][0]
    elif vecb[i]>domain[i][1]: vecb[i]=domain[i][1]

    # Calculate the current cost and the new cost
    ea=costf(vec)
    eb=costf(vecb)
    p=pow(math.e,(-eb-ea)/T)

    # Is it better, or does it make the probability
    # cutoff?
    if (eb<ea or random.random()<p):
      vec=vecb      

    # Decrease the temperature
    T=T*cool
  return vec


In [45]:
s1 = annealingoptimize(domain,schedulecost)
schedulecost(s1)
printschedule(s1)

   Seymour       BOS 13:40-15:37 $138 10:33-12:03 $ 74
    Franny       DAL 13:54-18:02 $294  9:49-13:51 $229
     Zooey       CAK 15:23-17:25 $232 10:32-13:16 $139
      Walt       MIA 15:34-18:11 $326  8:23-11:07 $143
     Buddy       ORD 14:22-16:32 $126  7:50-10:08 $164
       Les       OMA  9:15-12:03 $ 99  8:04-10:59 $136


In [14]:
domain

[(0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8)]

In [7]:
s = [1,4,3,2,7,3,6,3,2,4,5,3]
s = [1,4,3,2]
printschedule(s)
schedulecost(s)

schedulecost([3.0, 8.0, 0.0, 4.0, 6.0, 6.0, 0.0, 0.0, 5.0, 3.0, 6.0, 5.0])

   Seymour       BOS  8:04-10:11 $ 95 12:08-14:05 $142
    Franny       DAL 10:30-14:57 $290  9:49-13:51 $229


6125

In [8]:
domain = [(0,8)] * ( len(people) * 2)
randomoptimize(domain,schedulecost)

[4.0, 4.0, 0.0, 7.0, 1.0, 4.0, 4.0, 7.0, 4.0, 8.0, 4.0, 0.0]

In [9]:
flights[('LGA', 'BOS')]

[('6:39', '8:09', 86),
 ('8:23', '10:28', 149),
 ('9:58', '11:18', 130),
 ('10:33', '12:03', 74),
 ('12:08', '14:05', 142),
 ('13:39', '15:30', 74),
 ('15:25', '16:58', 62),
 ('17:03', '18:03', 103),
 ('18:24', '20:49', 124),
 ('19:58', '21:23', 142)]

In [10]:
s= hillclimb(domain, schedulecost)
schedulecost(s)

4855

In [11]:
sol=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
sol

[0, 2, 4, 1, 4, 0, 6, 8, 3, 7, 8, 4]

In [12]:
sol[0:5] + [sol[5] + 1] + sol[5+1:]
domain

[(0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8),
 (0, 8)]